In [1]:
from bookimed_lib import *

In [19]:
FEATURES = [
    "deg",
    "top", 
    "illn", 
    #"lang",
#    "ad_spec",
    "h_index",
    "exp_doc", 
    ]

In [68]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    for doc in all_doc:
        if len(doc['illnesses'])>0:
            X+=[[]]
            X[-1]+=[doc['degree'].count(u'Профессор')]    #doctor_prof_count
            X[-1]+=[int(doc['top'])]
            X[-1]+=[len(doc['illnesses'])]
            #X[-1]+=[0 if len(doc['illnesses'])>4 else 0]#len(doc['illnesses']) ]
            #X[-1]+=[len(doc['languages'])]
            #X[-1]+=[0]
            #if len(doc['languages']) > 0:
            #    lang=[int(k['id']) for k in doc['languages']]
            #    if 1 in lang:
            #        X[-1][-1]+=0.8
            #    if 2 in lang:
            #        X[-1][-1]+=0.2
            X[-1]+=[int(doc['h_index'])]
            #X[-1] +=[1 if len(doc['h_index'])>20 else 0 ]            
            X[-1]+=[int(doc['experience'])]
    y=[int(clinic_estim['specialist'])] * len(X)
    return [X, y]

In [4]:
def normale(mat):
    #for i in [2,4,5]:#lang
    for i in [2,3,4]:#no lang
        mass = [k[i] for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)
    #return mat

In [5]:
#2
def normale(mat):
    for i in range(0,len(mat[0])):
        mass = [k[i] for k in mat]
        mean = np.mean(mass)
        mx = max(mass)
        mn = min(mass)
        if mx!=mn:
            for j in mat:
                #print j[i], j[i] - mn, mx-mn
                j[i] = round((j[i] - mn) / float(mx-mn), 4)
                #print j[i]

In [28]:
def process_with(X,y, info=False, short=False, return_short = False, return_long = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=True, normalize=False)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print "%.3f" % np.mean(abs(regr.predict(train_X) - train_y))
        print "%.3f" % np.mean(abs(regr.predict(test_X) - test_y))
        print "%.3f" % (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100)
        print "%.3f" % regr.score(train_X, train_y)
        print "%.3f" % regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_long:
        return {"absolute_train": np.mean(abs(regr.predict(train_X) - train_y)),
               "absolute_test": np.mean(abs(regr.predict(test_X) - test_y)),
               "variance_train": regr.score(train_X, train_y),
               "variance_test": regr.score(test_X, test_y),
               ""
               "ts": ts}
    #for i in regr.coef_:
        #print "%.3f" % i
    #print "%.3f" % regr.intercept_
    return regr


In [69]:
X_all, X_1, X_2, X_3, X_4, X_5, y_all, y_1, y_2, y_3, y_4, y_5 = get_X_sets(extract_data)
normale(X_all)
#normale(X_1)
#normale(X_2)
#normale(X_3)
#normale(X_4)
#normale(X_5)

In [70]:
for i in range(len(X_all)-1,-1,-1):
    if X_all[i][-1]==0 or X_all[i][-2]==0 or X_all[i][2]==0:
        X_all.pop(i)
        y_all.pop(i)

In [71]:
len(X_all)

32

In [67]:
for i in range(1,10):
    print i
    regr = process_with(X_all, y_all, info=True, ts=i/10.0)
    print regr.coef_
    print

1
Total: 63, train: 56, test: 7
Residual sum of squares: 0.66
Train absolute: 0.53
Test absolute: 0.55
Absolute to mean: 6.34%
Train variance score: 0.27
Test variance score: 0.15
[ -5.69980158e-02   5.31052648e-01   1.12242081e-17   9.11521450e-01
   8.34201357e-01]

2
Total: 63, train: 50, test: 13
Residual sum of squares: 0.52
Train absolute: 0.56
Test absolute: 0.51
Absolute to mean: 6.11%
Train variance score: 0.26
Test variance score: 0.24
[ -6.70913857e-02   5.16324033e-01  -9.76080267e-16   1.12048240e+00
   6.08245066e-01]

3
Total: 63, train: 44, test: 19
Residual sum of squares: 0.53
Train absolute: 0.55
Test absolute: 0.50
Absolute to mean: 6.04%
Train variance score: 0.29
Test variance score: 0.17
[ -5.58301236e-02   7.64477930e-01   1.32182980e-16   9.06542376e-01
   5.89725684e-01]

4
Total: 63, train: 37, test: 26
Residual sum of squares: 0.59
Train absolute: 0.55
Test absolute: 0.53
Absolute to mean: 6.34%
Train variance score: 0.31
Test variance score: 0.16
[ -4.49707

In [73]:
regr = process_with(X_all, y_all, info=True, ts=0.4)

Total: 32, train: 19, test: 13
Residual sum of squares: 1.12
Train absolute: 0.50
Test absolute: 0.78
Absolute to mean: 8.86%
Train variance score: 0.57
Test variance score: -0.24


In [75]:
regr.coef_

array([ 0.34073574,  0.63752363,  0.28220311,  0.38026313,  1.07172097])

In [76]:
print regr.intercept_

7.10154227025


In [62]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y_2)):
    print "Real: %f \t Predicted: %f" %(y_2[i], regr.predict(X_2[i]))

Real: 10.000000 	 Predicted: 7.791874
Real: 10.000000 	 Predicted: 8.217326
Real: 10.000000 	 Predicted: 7.907875
Real: 10.000000 	 Predicted: 8.741323
Real: 10.000000 	 Predicted: 7.698840
Real: 10.000000 	 Predicted: 7.767454
Real: 10.000000 	 Predicted: 8.760110
Real: 10.000000 	 Predicted: 8.926070
Real: 10.000000 	 Predicted: 7.818978
Real: 9.000000 	 Predicted: 7.620211
Real: 9.000000 	 Predicted: 7.620211
Real: 9.000000 	 Predicted: 7.620211
Real: 9.000000 	 Predicted: 9.240602
Real: 9.000000 	 Predicted: 8.065396
Real: 9.000000 	 Predicted: 8.331904
Real: 9.000000 	 Predicted: 7.729460
Real: 9.000000 	 Predicted: 7.860400
Real: 8.000000 	 Predicted: 8.727321
Real: 8.000000 	 Predicted: 8.467163
Real: 8.000000 	 Predicted: 8.069797
Real: 8.000000 	 Predicted: 7.912538
Real: 8.000000 	 Predicted: 7.833909
Real: 8.000000 	 Predicted: 8.722285
Real: 8.000000 	 Predicted: 7.856099
Real: 8.000000 	 Predicted: 7.620211
Real: 8.000000 	 Predicted: 7.975571
Real: 8.000000 	 Predicted: 8

In [74]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print len(FEATURES)
for i in f:
    print "%4f \t %s" %(i[0], i[1])
    #print i[0]
print regr.intercept_

Features sorted by their score:
5
0.340700 	 deg
0.637500 	 top
0.282200 	 illn
0.380300 	 h_index
1.071700 	 exp_doc
7.10154227025


In [ ]:
f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [ ]:
plt.plot(sorted([i[-2] for i in X_all]))
plt.show()

In [16]:
plt.scatter([i[3] for i in X_all], y_all)
plt.show()

In [ ]:
plt.scatter([i[2] for i in X_all], [i[3] for i in X_all])
plt.show()

In [61]:
for i in range(0, len(X_all)):
    print y_all[i], X_all[i][-2]

10 0.3462
10 0.6538
8 0.0577
8 0.1154
10 0.0769
7 0.0192
8 0.0577
8 0.2115
8 0.25
8 0.2115
8 0.0385
8 0.0385
10 0.3269
10 0.0385
10 0.6538
8 0.4231
8 0.3462
8 0.1538
8 0.4423
8 0.6154
8 0.4231
8 0.3846
7 0.2692
7 0.0385
8 0.0192
8 0.2308
9 0.5769
8 0.5192
8 0.2308
8 0.3462
8 0.0769
10 0.8846
8 0.1154
8 0.1731
8 0.2115
8 0.0769
8 0.3077
8 0.1923
9 0.0385
9 0.0385
9 0.1923
8 0.3077
8 0.2885
8 0.2692
8 0.1154
8 0.2692
8 0.0385
8 0.25
